In [3]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt 
import string
import os
from PIL import Image
import glob
from pickle import dump, load
from tqdm import tqdm_notebook as tqdm
from time import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

Using TensorFlow backend.


In [4]:
try:
    import dill as pickle
except ImportError:
    import pickle

In [5]:
# Loading Image Features
train_features = load(open("C:/Flickr8k_urdu_17july/encoded_train_images_17july.pkl", "rb"))
print('Photos: train=%d' % len(train_features))

Photos: train=6000


In [6]:
# Load Train Descriptions, which contains, 5 captions corresponding to a key
train_descriptions = load(open("C:/Flickr8k_urdu_17july/train_descriptions_17july.pkl", "rb"))
print('Photos: train=%d' % len(train_descriptions))

Photos: train=6000


In [7]:
# Create a list of all the training captions
train_captions = []
for key, val in train_descriptions.items():
    for cap in val:
        train_captions.append(cap)
len(train_captions)

30000

In [8]:
threshold = 7
word_counts = {}
nsents = 0
for sent in train_captions:
    nsents += 1
    for w in sent.split(' '):
        if w not in word_counts:
            word_counts[w] = 0
        word_counts[w] += 1

vocab = [w for w in word_counts if word_counts[w] >= threshold]
print('preprocessed words %d -> %d' % (len(word_counts), len(vocab)))


preprocessed words 6936 -> 2144


In [9]:
ixtoword = {}
wordtoix = {}

ix = 1
for w in vocab:
    wordtoix[w] = ix
    ixtoword[ix] = w
    ix += 1

vocab_size = len(ixtoword) + 1 # one for appended 0's
vocab_size

2145

In [10]:
vocab_size = len(ixtoword) + 1 # one for appended 0's
vocab_size

2145

In [11]:
# calculate the length of the description with the most words
def max_length(descriptions): 
    max_l = -1e9
    for key, cap in descriptions.items():
        for line in cap:
            max_l = max(max_l, len(line.split(' ')))
    return max_l

max_length = max_length(train_descriptions)
print('Description Length: %d' % max_length)

Description Length: 48


In [12]:
# Load Glove vectors, which contains vectors(200 Dimension) corresponding to words. 
glove_dir = 'C:/Flickr8k_urdu_17july/Word2Vec'
embeddings_index = {}
f = open(os.path.join(glove_dir, 'embedding_vec_urdu_17july.txt'), encoding="utf-8")

for line in tqdm(f):
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))

C:\Users\HP\anaconda3\envs\python_36\lib\site-packages\ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


0it [00:00, ?it/s]

Found 64654 word vectors.


In [13]:
embedding_dim = 300

embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in tqdm(wordtoix.items()):
    if word in embeddings_index:
        embedding_vector = embeddings_index[word]
        embedding_matrix[i] = embedding_vector

C:\Users\HP\anaconda3\envs\python_36\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/2144 [00:00<?, ?it/s]

ERROR! Session/line number was not unique in database. History logging moved to new session 325


In [14]:
embedding_matrix.shape # (vocab, embedding_dim)

(2145, 300)

In [15]:
with open('C:/Flickr8k_urdu_17july/Embedding_Matrix_17july.pkl', 'wb') as embedding_pickle:
    pickle.dump(embedding_matrix, embedding_pickle)

In [16]:
with open('C:/Flickr8k_urdu_17july/wordtoix_17july.pkl', 'wb') as wti_pickle:
    pickle.dump(wordtoix, wti_pickle)